<a href="https://colab.research.google.com/github/JEN12H/House-Price-Prediction/blob/main/House_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTING ALL NECESSARY IMPORTS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# BASIC PRE-PROCESSING

In [ ]:
df = pd.read_csv('/content/train.csv')

# Drop columns
columns_to_drop = ['Alley','PoolQC','MiscFeature','Fence','FireplaceQu','RoofMatl','Street','BsmtFinType2','BsmtFinSF1','BsmtFinSF2','2ndFlrSF','1stFlrSF','Heating','TotRmsAbvGrd','MiscVal','Condition1','Condition2','YearRemodAdd','MasVnrType','MasVnrArea','ScreenPorch','3SsnPortch','EnclosedPorch','OpenPorchSF','PavedDrive','GarageCond','HalfBath','BsmtHalfBath','LowQualFinSF','BsmtUnfSF','BsmtExposure','PoolArea','WoodDeckSF']
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
df_clean = df.drop(columns=existing_columns_to_drop, axis=1)

# Fill missing 'LotFrontage' values with the mean (using the mean from the original training data)
df_clean['LotFrontage'] = df_clean['LotFrontage'].fillna(df_clean['LotFrontage'].mean())


display(df_clean.head())

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageFinish,GarageCars,GarageArea,GarageQual,3SsnPorch,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Reg,Lvl,AllPub,Inside,Gtl,...,RFn,2,548,TA,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Reg,Lvl,AllPub,FR2,Gtl,...,RFn,2,460,TA,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,IR1,Lvl,AllPub,Inside,Gtl,...,RFn,2,608,TA,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,IR1,Lvl,AllPub,Corner,Gtl,...,Unf,3,642,TA,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,IR1,Lvl,AllPub,FR2,Gtl,...,RFn,3,836,TA,0,12,2008,WD,Normal,250000


In [ ]:
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].mean())
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

None

In [ ]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
df_clean.info()
df_clean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 49 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  BldgType       1460 non-null   object 
 12  HouseStyle     1460 non-null   object 
 13  OverallQual    1460 non-null   int64  
 14  OverallCond    1460 non-null   int64  
 15  YearBuilt      1460 non-null   int64  
 16  RoofStyle      1460 non-null   object 
 17  Exterior1st    1460 non-null   object 
 18  Exterior

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageFinish,GarageCars,GarageArea,GarageQual,3SsnPorch,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Reg,Lvl,AllPub,Inside,Gtl,...,RFn,2,548,TA,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Reg,Lvl,AllPub,FR2,Gtl,...,RFn,2,460,TA,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,IR1,Lvl,AllPub,Inside,Gtl,...,RFn,2,608,TA,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,IR1,Lvl,AllPub,Corner,Gtl,...,Unf,3,642,TA,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,IR1,Lvl,AllPub,FR2,Gtl,...,RFn,3,836,TA,0,12,2008,WD,Normal,250000


# HANDLING IMBALANCE COLUMNS

In [ ]:
# Apply the same replacements for categorical features
df_clean['Functional'] = df_clean['Functional'].replace(['Min2','Min1','Maj1','Maj2','Mod','Sev'],('Other'))
df_clean['Functional'].value_counts()
df_clean['Electrical'] = df_clean['Electrical'].replace(['Mix','FuseP','FuseF','FuseA'],('Other'))
df_clean['Electrical']
df_clean['BsmtCond'] = df_clean['BsmtCond'].replace(['Gd','Fa','Po'],('Other'))
df_clean['BsmtCond']
df_clean['LandContour'] = df_clean['LandContour'].replace(['Bnk','HLS','Low'],('Other'))
df_clean['LandContour']
df_clean = df_clean.dropna()
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1348 entries, 0 to 1459
Data columns (total 49 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1348 non-null   int64  
 1   MSSubClass     1348 non-null   int64  
 2   MSZoning       1348 non-null   object 
 3   LotFrontage    1348 non-null   float64
 4   LotArea        1348 non-null   int64  
 5   LotShape       1348 non-null   object 
 6   LandContour    1348 non-null   object 
 7   Utilities      1348 non-null   object 
 8   LotConfig      1348 non-null   object 
 9   LandSlope      1348 non-null   object 
 10  Neighborhood   1348 non-null   object 
 11  BldgType       1348 non-null   object 
 12  HouseStyle     1348 non-null   object 
 13  OverallQual    1348 non-null   int64  
 14  OverallCond    1348 non-null   int64  
 15  YearBuilt      1348 non-null   int64  
 16  RoofStyle      1348 non-null   object 
 17  Exterior1st    1348 non-null   object 
 18  Exterior2nd  

#SEGGREGATING COLUMNS FOR FURTHER PROCESSING

In [ ]:
ordinal_cols = ['LotShape','KitchenQual','CentralAir',
                'HeatingQC','BsmtQual','Utilities','LandSlope','HouseStyle',
                'ExterQual','ExterCond','SaleCondition','GarageQual',
                'GarageFinish','Functional','LandContour']
numerical_cols = ['TotalBsmtSF','LotFrontage','LotArea','GrLivArea','','GarageArea']
LabelEnco_cols = ['GarageQual','MSZoning','SaleType','LotConfig',
                  'Neighborhood','BldgType','RoofStyle','Exterior1st','Exterior2nd',
                  'Foundation','Electrical','BsmtCond']

In [ ]:
# Calculate the age of the houses and garages
current_year = 2025
df_clean['YearBuilt'] = current_year - df_clean['YearBuilt']
df_clean['GarageYrBlt'] = current_year - df_clean['GarageYrBlt']

In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1348 entries, 0 to 1459
Data columns (total 49 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1348 non-null   int64  
 1   MSSubClass     1348 non-null   int64  
 2   MSZoning       1348 non-null   object 
 3   LotFrontage    1348 non-null   float64
 4   LotArea        1348 non-null   int64  
 5   LotShape       1348 non-null   object 
 6   LandContour    1348 non-null   object 
 7   Utilities      1348 non-null   object 
 8   LotConfig      1348 non-null   object 
 9   LandSlope      1348 non-null   object 
 10  Neighborhood   1348 non-null   object 
 11  BldgType       1348 non-null   object 
 12  HouseStyle     1348 non-null   object 
 13  OverallQual    1348 non-null   int64  
 14  OverallCond    1348 non-null   int64  
 15  YearBuilt      1348 non-null   int64  
 16  RoofStyle      1348 non-null   object 
 17  Exterior1st    1348 non-null   object 
 18  Exterior2nd  

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df_clean.drop('SalePrice',axis=1),df_clean['SalePrice'],test_size=0.25,random_state=42)

In [ ]:
trf1 = ColumnTransformer([('ordinal_encoded',OrdinalEncoder(),ordinal_cols)],remainder = 'passThrough')

# MODEL PREPARATION

In [ ]:
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from xgboost import XGBRegressor

# PIPELINE

In [ ]:
from sklearn.metrics import r2_score
base_pipeline = Pipeline([
    ('transformer',trf1),
    ('model',RandomForestRegressor())
])

model_params = {
    "RandomForest":{
        "model":RandomForestRegressor(random_state=42),
        'params':{
            'model__n_estimators':[50,100,150,200],
            'model__max_depth':[5,10,None]
        }
    },
    "AdaBoost":{
        "model":AdaBoostRegressor(random_state=42),
    "params":{
        'model__n_estimators':[50,100,150,200],
        'model__learning_rate':[0.01,0.1,0.2]
        }
    },

    "XGBoost":{
        "model":XGBRegressor(random_state=42,eval_metric ='rmse'),
        "params":{
           'model__n_estimators':[50,100,150,200],
           'model__learning_rate':[0.01,0.1,0.2]


        }
    }

}

best_model  ={}



for name,mp in model_params.items():
  print(f"Running for model {name}")

  pipe = Pipeline(steps=[
      ('transformer', ColumnTransformer([
          ('ordinal_encoded', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), X_train.select_dtypes(include='object').columns),
          ('scaled_numerical', StandardScaler(), X_train.select_dtypes(include=np.number).columns)
      ], remainder='passthrough')), # Corrected spelling here
      ('model',mp['model'])
       ])
  grid = GridSearchCV(
      estimator= pipe,
      param_grid = mp['params'],
      cv = 5,
      scoring = 'neg_mean_squared_error',
      n_jobs = 1
  )

  grid.fit(X_train,y_train)
  y_pred = grid.predict(X_test)
  rmse = np.sqrt(mean_squared_error(y_test,y_pred))
  r2 = r2_score(y_test,y_pred)
  print(f"BEST R2 SCORE {r2:.4f}")
  print(f"Best Params for {name}:{grid.best_params_}")
  print(f"CV Best Score(MSE):{grid.best_score_:.4f}")
  print(f"Test Rmse:{rmse:.4f}")
  best_model[name] = grid.best_estimator_

Running for model RandomForest
BEST R2 SCORE 0.8794
Best Params for RandomForest:{'model__max_depth': 10, 'model__n_estimators': 100}
CV Best Score(MSE):-1360048923.4810
Test Rmse:24931.4853
Running for model AdaBoost
BEST R2 SCORE 0.8053
Best Params for AdaBoost:{'model__learning_rate': 0.2, 'model__n_estimators': 200}
CV Best Score(MSE):-1689659494.8378
Test Rmse:31676.7348
Running for model XGBoost
BEST R2 SCORE 0.8869
Best Params for XGBoost:{'model__learning_rate': 0.2, 'model__n_estimators': 150}
CV Best Score(MSE):-1371069804.8000
Test Rmse:24145.0510


# TESTING-DATA PREPARATION

In [ ]:
fd = pd.read_csv('/content/test.csv')


columns_to_drop = ['Alley','PoolQC','MiscFeature','Fence','FireplaceQu','RoofMatl','Street','BsmtFinType2','BsmtFinSF1','BsmtFinSF2','2ndFlrSF','1stFlrSF','Heating','TotRmsAbvGrd','MiscVal','Condition1','Condition2','YearRemodAdd','MasVnrType','MasVnrArea','ScreenPorch','3SsnPortch','EnclosedPorch','OpenPorchSF','PavedDrive','GarageCond','HalfBath','BsmtHalfBath','LowQualFinSF','BsmtUnfSF','BsmtExposure','PoolArea','WoodDeckSF']
existing_columns_to_drop = [col for col in columns_to_drop if col in fd.columns]
fd_clean = fd.drop(columns=existing_columns_to_drop, axis=1)

# Fill missing 'LotFrontage' values with the mean (using the mean from the original training data)
# Note: Using the mean from the *training* data is important for consistency between train and test sets
fd_clean['LotFrontage'] = fd_clean['LotFrontage'].fillna(df['LotFrontage'].mean())

# Apply the same replacements for categorical features
fd_clean['Functional'] = fd_clean['Functional'].replace(['Min2','Min1','Maj1','Maj2','Mod','Sev'],('Other'))
fd_clean['Electrical'] = fd_clean['Electrical'].replace(['Mix','FuseP','FuseF','FuseA'],('Other'))
fd_clean['BsmtCond'] = fd_clean['BsmtCond'].replace(['Gd','Fa','Po'],('Other'))
fd_clean['LandContour'] = fd_clean['LandContour'].replace(['Bnk','HLS','Low'],('Other'))

# Calculate the age of the houses and garages
current_year = 2025
fd_clean['YearBuilt'] = current_year - fd_clean['YearBuilt']
fd_clean['GarageYrBlt'] = current_year - fd_clean['GarageYrBlt']

display(fd_clean.head())

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,3SsnPorch,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Reg,Lvl,AllPub,Inside,Gtl,...,64.0,Unf,1.0,730.0,TA,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,IR1,Lvl,AllPub,Corner,Gtl,...,67.0,Unf,1.0,312.0,TA,0,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,IR1,Lvl,AllPub,Inside,Gtl,...,28.0,Fin,2.0,482.0,TA,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,IR1,Lvl,AllPub,Inside,Gtl,...,27.0,Fin,2.0,470.0,TA,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,IR1,Other,AllPub,Inside,Gtl,...,33.0,RFn,2.0,506.0,TA,0,1,2010,WD,Normal


In [ ]:
df_clean.info()
fd_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1348 entries, 0 to 1459
Data columns (total 49 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1348 non-null   int64  
 1   MSSubClass     1348 non-null   int64  
 2   MSZoning       1348 non-null   object 
 3   LotFrontage    1348 non-null   float64
 4   LotArea        1348 non-null   int64  
 5   LotShape       1348 non-null   object 
 6   LandContour    1348 non-null   object 
 7   Utilities      1348 non-null   object 
 8   LotConfig      1348 non-null   object 
 9   LandSlope      1348 non-null   object 
 10  Neighborhood   1348 non-null   object 
 11  BldgType       1348 non-null   object 
 12  HouseStyle     1348 non-null   object 
 13  OverallQual    1348 non-null   int64  
 14  OverallCond    1348 non-null   int64  
 15  YearBuilt      1348 non-null   int64  
 16  RoofStyle      1348 non-null   object 
 17  Exterior1st    1348 non-null   object 
 18  Exterior2nd  

In [ ]:
fd_clean.shape

(1459, 48)

In [ ]:
df_clean.shape

(1348, 49)

In [ ]:
X_train = df_clean.drop('SalePrice',axis = 1)
y_train = df_clean['SalePrice']

X_test = fd_clean.copy()
y_pred = best_model['XGBoost'].predict(X_test)

fd_clean['SalePrice'] = y_pred


In [ ]:
fd_clean.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageFinish,GarageCars,GarageArea,GarageQual,3SsnPorch,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,RH,80.0,11622,Reg,Lvl,AllPub,Inside,Gtl,...,Unf,1.0,730.0,TA,0,6,2010,WD,Normal,127948.125000
1,1462,20,RL,81.0,14267,IR1,Lvl,AllPub,Corner,Gtl,...,Unf,1.0,312.0,TA,0,6,2010,WD,Normal,153666.281250
2,1463,60,RL,74.0,13830,IR1,Lvl,AllPub,Inside,Gtl,...,Fin,2.0,482.0,TA,0,3,2010,WD,Normal,178220.031250
3,1464,60,RL,78.0,9978,IR1,Lvl,AllPub,Inside,Gtl,...,Fin,2.0,470.0,TA,0,6,2010,WD,Normal,187241.421875
4,1465,120,RL,43.0,5005,IR1,Other,AllPub,Inside,Gtl,...,RFn,2.0,506.0,TA,0,1,2010,WD,Normal,191968.734375


# FINAL-FILE WITH PREDICTIONS

In [ ]:
fd_clean.to_csv('submission.csv',index=False)